## Setup

For these examples, you'll need:

1. qiskit, version >=1.0 and <2.0
2. qiskit-ionq
3. An IonQ API key

If this notebook was not launched from an environment where Qiskit and Qiskit-IonQ are installed, uncomment and run the next cell to install them in this notebook's kernel.

In [ ]:
#%pip install "qiskit==1.4" qiskit-ionq

Set your API key as an environment variable from here, if needed:

In [ ]:
import os
os.environ["IONQ_API_KEY"] = "YOUR API KEY HERE"

Set up the IonQProvider for Qiskit:

In [ ]:
from qiskit_ionq import IonQProvider
provider = IonQProvider()

Get backends for the ideal simulator and for Aria:

In [ ]:
backend_sim = provider.get_backend("simulator")
backend_aria = provider.get_backend("qpu.aria-1")

# Error mitigation: debiasing and sharpening

Debiasing and sharpening are only currently available on our QPUs, not our simulators or noisy simulators.

Debiasing is a job-level setting _on by default_ for jobs with 500 or more shots on any Aria or Forte QPU, but can be turned off. It is _off by default_ for jobs with fewer than 500 shots, and cannot be turned on.

Sharpening is an option used for result retrieval; by default, result aggregation uses averaging (i.e., `sharpening=False`) but sharpening is an available option for any job run with debiasing. 

## Debiasing (symmetrization)

It's not easy to contrive a reproducible example that clearly shows the effect of debiasing - the error rates on our systems are relatively low, some errors change over time, and only some of the error is the systematic error addressed by debiasing.

The more qubits and gate types we use, the more likely we will encounter the types of errors that are mitigated by debiasing. Let's make a GHZ state using 18 qubits:

In [ ]:
from qiskit import QuantumCircuit
n = 18

qc = QuantumCircuit(n, name=f"GHZ state, {n} qubits")
qc.h(0)
for i in range(1,n):
    qc.cx(0, i)
qc.measure_all()

Run it on the ideal simulator, and submit to Aria with and without debiasing. We use `error_mitigation=ErrorMitigation.DEBIASING` to explicitly set this option in Qiskit.

(Since debiasing is on by default for jobs with at least 500 shots, we don't need to explicitly turn it on for the debiased job here, but we'll show it for completeness.)

In [ ]:
from qiskit_ionq import ErrorMitigation

In [ ]:
job1_ideal = backend_sim.run(qc, shots=2500)

job1_debiasing_on = backend_aria.run(
    qc,
    shots=2500,
    error_mitigation=ErrorMitigation.DEBIASING
)

job1_debiasing_off = backend_aria.run(
    qc,
    shots=2500,
    error_mitigation=ErrorMitigation.NO_DEBIASING
)

print("Job IDs:")
print(f"Ideal simulation: {job1_ideal.job_id()}")
print(f"Aria with debiasing on: {job1_debiasing_on.job_id()}")
print(f"Aria with debiasing off: {job1_debiasing_off.job_id()}")

Next, wait for these jobs to complete.

In [ ]:
print(job1_debiasing_on.status())
print(job1_debiasing_off.status())

Wait for the jobs to complete.

Optional: retrieve jobs by their IDs (printed above or copied from the cloud console) if you needed to close this notebook and come back later.

In [ ]:
#job1_ideal = sim_backend.retrieve_job("PASTE JOB ID HERE")
#job1_on = aria_backend.retrieve_job("PASTE JOB ID HERE")
#job1_off = aria_backend.retrieve_job("PASTE JOB ID HERE")

For the QPU jobs, these are large histograms with many states, so we won't display all of the states. We're looking for the top two highest-probability states for each job, which should be the 0-state and the 1-state.

In [ ]:
from qiskit.visualization import plot_histogram

In [ ]:
# ! don't run this until the jobs have completed

plot_histogram(
    [job1_ideal.get_counts(), job1_debiasing_on.get_counts(), job1_debiasing_off.get_counts()],
    legend=["Ideal simulation", "Aria, debiasing on", "Aria, debiasing off"],
    figsize=(8,3),
    number_to_keep=2
)

## Averaging vs sharpening

For a job that was run with debiasing, you can aggregate the results from the different executions in two different ways

Let's submit another job with debiasing on. This is a smaller circuit, but it yields a quantum state with different relative probabilities for several basis states.

In [ ]:
qc2 = QuantumCircuit(3, name="Sharpening example")
qc2.rx(3.14159/3, 0)
qc2.rx(3.14159/3, 1)
qc2.rx(3.14159/3, 2)
qc2.cx(0, 1)
qc2.cx(0, 2)
qc2.measure_all()

In [ ]:
job2_ideal = backend_sim.run(qc2)
print(job2_ideal.job_id())

In [ ]:
job2_debiasing_on = backend_aria.run(
    qc2,
    shots=2500,
    error_mitigation=ErrorMitigation.DEBIASING
)

print(job2_debiasing_on.job_id())

While we wait for job #2 to run, let's go back and look at job #1, the GHZ state prep circuit we ran earlier.

This job should have two states with equal probabilities, but on the actual QPU there were some results measured for other states. This happens even when we're using debiasing to run multiple circuit variants - debiasing reduces or removes some types of systematic error, but doesn't mitigate all types of error.

In [ ]:
plot_histogram(
    job1_on.get_counts(),
    figsize=(10,3),
    number_to_keep=50
)

If we're trying to identify the highest-probability state or states, or we know that low-probability states only result from errors, we can aggregate the results via _sharpening_.

In [ ]:
plot_histogram(
    [job1_on.get_counts(), job1_on.result(sharpen=True).get_counts()],
    legend=["Debiasing + averaging", "Debiasing + sharpening"],
    figsize=(10,3),
    number_to_keep=50
)

Here's what it looks like if we just plot the 0 and 1 states, and everything else:

In [ ]:
plot_histogram(
    [job1_on.get_counts(), job1_on.result(sharpen=True).get_counts()],
    legend=["Debiasing + averaging", "Debiasing + sharpening"],
    figsize=(10,3),
    number_to_keep=2
)

All of the measurements associated with errors were removed from the result when we aggregate with sharpening: within each execution (group of shots), either the 0-state or the 1-state was always the clear winner of plurality voting, so all shots were assigned to one of those states.

However, we may or may not have ended up with the expected distribution between the 0-state and the 1-state. So if our algorithm or application was based on successfully _identifying_ these two states, sharpening would help amplify them, but if we need their relative probabilities, sharpening would likely distort the distribution.

Note that if you request sharpening and there is no clear winner of the plurality voting for a particular execution, that execution's results will be returned via averaging instead. This reduces the potential for sharpening to distort some types of very complex probability distributions, but it is still important to consider whether this type of aggregation makes sense for your application.

Let's look at the second job, with the smaller circuit:

In [ ]:
# Multipy the ideal probabilities by the number of shots instead of sampling
ideal_counts = {key: int(val*2500) for key, val in job2_ideal.get_probabilities().items()}

counts_with_averaging = job2_debiasing_on.result(sharpen=False).get_counts()
# This is equivalent, because sharpen=False by default:
# counts_with_averaging = job_debiasing_on.get_counts()

counts_with_sharpening = job2_debiasing_on.result(sharpen=True).get_counts()

In [ ]:
plot_histogram(
    [ideal_counts, counts_with_averaging, counts_with_sharpening],
    legend=["Ideal", "Averaging", "Sharpening"],
    figsize=(10,3),
    bar_labels=False
)

All of the shots from the low-probability states were absorbed into the 000 state by sharpening! If this result was for an optimization where we're trying to find the best solution (highest-probability state), this would amplify the probability for that state. However, if we needed the relative probabilities of all of the states, we'd be better off averaging.

More about debiasing and sharpening
* [Enhancing quantum computer performance via symmetrization](https://arxiv.org/abs/2301.07233) on arXiv
* [Debiasing and sharpening guide](https://ionq.com/resources/debiasing-and-sharpening) in our resource center
* [Syntax for debiasing and sharpening with ionq-qiskit](https://docs.ionq.com/sdks/qiskit/error-mitigation-qiskit) in our docs